In [1]:
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Layer, Input, concatenate, Conv1D, MaxPooling1D, Conv2DTranspose, Lambda, \
    BatchNormalization, Bidirectional, LSTM, Dropout, Dense, InputLayer, Conv2D, MaxPooling2D, Flatten,\
    AveragePooling2D, GlobalAveragePooling2D, GlobalAveragePooling1D, AveragePooling1D, MultiHeadAttention,\
    LayerNormalization, Embedding, LeakyReLU, Conv1DTranspose

2023-06-04 02:01:24.769444: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-04 02:01:24.810780: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-04 02:01:25.338691: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
N = 12500
N_TRAIN = 10000
N_VAL = 1000
N_TEST = 1500

y_train = np.concatenate([np.zeros((N_TRAIN, 1, 1), dtype=int), np.ones((N_TRAIN, 1, 1), dtype=int)])
y_val = np.concatenate([np.zeros((N_VAL, 1, 1), dtype=int), np.ones((N_VAL, 1, 1), dtype=int)])
y_test = np.concatenate([np.zeros((N_TEST, 1, 1), dtype=int), np.ones((N_TEST, 1, 1), dtype=int)])


In [3]:
cat = np.reshape(np.load("cat_hog.npy"), (N, 1, 72900))
dog = np.reshape(np.load("dog_hog.npy"), (N, 1, 72900))

x_train = np.concatenate([cat[:N_TRAIN], dog[:N_TRAIN]])
x_val = np.concatenate([cat[N_TRAIN:N_TRAIN+N_VAL], dog[N_TRAIN:N_TRAIN+N_VAL]])
x_test = np.concatenate([cat[-N_TEST:], dog[-N_TEST:]])

model = Sequential([
    InputLayer(input_shape=(1, 72900)),
    # Conv1D(1024, 5, padding='same', activation='relu'),
    # Conv1D(512, 11, padding='same', activation='relu'),
    # Conv1D(256, 17, padding='same', activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.optimizers.Adam(0.001),
              loss=tf.keras.losses.binary_crossentropy,
              metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

model.summary()

model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=100, batch_size=500,
          callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)])

print(model.evaluate(x_test, y_test, batch_size=500, return_dict=True))

model.save("model")

del cat, dog, x_train, x_val, x_test, model

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1, 1)              72901     
                                                                 
Total params: 72,901
Trainable params: 72,901
Non-trainable params: 0
_________________________________________________________________


2023-06-04 02:01:40.199998: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/100
40/40 [==============================] - 4s 75ms/step - loss: 0.6686 - binary_accuracy: 0.6265 - precision: 0.6243 - recall: 0.6350 - val_loss: 0.5521 - val_binary_accuracy: 0.7205 - val_precision: 0.7309 - val_recall: 0.6980
Epoch 2/100
40/40 [==============================] - 1s 19ms/step - loss: 0.5298 - binary_accuracy: 0.7461 - precision: 0.7440 - recall: 0.7503 - val_loss: 0.5336 - val_binary_accuracy: 0.7365 - val_precision: 0.7846 - val_recall: 0.6520
Epoch 3/100
40/40 [==============================] - 1s 20ms/step - loss: 0.5069 - binary_accuracy: 0.7609 - precision: 0.7560 - recall: 0.7704 - val_loss: 0.5250 - val_binary_accuracy: 0.7405 - val_precision: 0.7833 - val_recall: 0.6650
Epoch 4/100
40/40 [==============================] - 1s 17ms/step - loss: 0.4886 - binary_accuracy: 0.7723 - precision: 0.7708 - recall: 0.7751 - val_loss: 0.5087 - val_binary_accuracy: 0.7575 - val_precision: 0.7614 - val_recall: 0.7500
Epoch 5/100
40/40 [=============================

2023-06-04 02:02:07.439470: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_1' with dtype float and shape [?,1,72900]
	 [[{{node input_1}}]]
2023-06-04 02:02:07.444500: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_1' with dtype float and shape [?,1,72900]
	 [[{{node input_1}}]]
2023-06-04 02:02:07.449871: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,1,72900]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets


In [4]:
x = np.reshape(np.load("pearson.npy"), (N << 1, 1, 50301))
cat, dog = x[:N], x[N:]

x_train = np.concatenate([cat[:N_TRAIN], dog[:N_TRAIN]])
x_val = np.concatenate([cat[N_TRAIN:N_TRAIN+N_VAL], dog[N_TRAIN:N_TRAIN+N_VAL]])
x_test = np.concatenate([cat[-N_TEST:], dog[-N_TEST:]])

model = Sequential([
    InputLayer(input_shape=(1, 50301)),
    # Conv1D(1024, 5, padding='same', activation='relu'),
    # Conv1D(512, 11, padding='same', activation='relu'),
    # Conv1D(256, 17, padding='same', activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.optimizers.Adam(0.001),
              loss=tf.keras.losses.binary_crossentropy,
              metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

model.summary()

model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=100, batch_size=500,
          callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)])

print(model.evaluate(x_test, y_test, batch_size=500, return_dict=True))

model.save("model_pearson")

del x, cat, dog, x_train, x_val, x_test, model

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 1, 1)              50302     
                                                                 
Total params: 50,302
Trainable params: 50,302
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
40/40 [==============================] - 2s 51ms/step - loss: 0.6643 - binary_accuracy: 0.6270 - precision_1: 0.6216 - recall_1: 0.6493 - val_loss: 0.5935 - val_binary_accuracy: 0.7100 - val_precision_1: 0.6868 - val_recall_1: 0.7720
Epoch 2/100
40/40 [==============================] - 1s 19ms/step - loss: 0.5673 - binary_accuracy: 0.7196 - precision_1: 0.7128 - recall_1: 0.7355 - val_loss: 0.5661 - val_binary_accuracy: 0.7255 - val_precision_1: 0.7589 - val_recall_1: 0.6610
Epoch 3/100
40/40 [==============================] - 1s 14ms/step - loss: 0.5381 - bin

2023-06-04 02:02:38.925433: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_2' with dtype float and shape [?,1,50301]
	 [[{{node input_2}}]]
2023-06-04 02:02:38.933052: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_2' with dtype float and shape [?,1,50301]
	 [[{{node input_2}}]]
2023-06-04 02:02:38.941947: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,1,50301]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: model_pearson/assets


INFO:tensorflow:Assets written to: model_pearson/assets


In [5]:
x = np.reshape(np.load("chi_square.npy"), (N << 1, 1, 50301))
cat, dog = x[:N], x[N:]

x_train = np.concatenate([cat[:N_TRAIN], dog[:N_TRAIN]])
x_val = np.concatenate([cat[N_TRAIN:N_TRAIN+N_VAL], dog[N_TRAIN:N_TRAIN+N_VAL]])
x_test = np.concatenate([cat[-N_TEST:], dog[-N_TEST:]])

model = Sequential([
    InputLayer(input_shape=(1, 50301)),
    # Conv1D(1024, 5, padding='same', activation='relu'),
    # Conv1D(512, 11, padding='same', activation='relu'),
    # Conv1D(256, 17, padding='same', activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.optimizers.Adam(0.001),
              loss=tf.keras.losses.binary_crossentropy,
              metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

model.summary()

model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=100, batch_size=500,
          callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)])

print(model.evaluate(x_test, y_test, batch_size=500, return_dict=True))

model.save("model_chi_square")

del x, cat, dog, x_train, x_val, x_test, model

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 1, 1)              50302     
                                                                 
Total params: 50,302
Trainable params: 50,302
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
40/40 [==============================] - 2s 46ms/step - loss: 0.7413 - binary_accuracy: 0.5940 - precision_2: 0.5937 - recall_2: 0.5957 - val_loss: 0.5664 - val_binary_accuracy: 0.7100 - val_precision_2: 0.7225 - val_recall_2: 0.6820
Epoch 2/100
40/40 [==============================] - 1s 13ms/step - loss: 0.5439 - binary_accuracy: 0.7362 - precision_2: 0.7342 - recall_2: 0.7403 - val_loss: 0.5390 - val_binary_accuracy: 0.7330 - val_precision_2: 0.7161 - val_recall_2: 0.7720
Epoch 3/100
40/40 [==============================] - 1s 15ms/step - loss: 0.5208 - bin

2023-06-04 02:03:07.444098: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_3' with dtype float and shape [?,1,50301]
	 [[{{node input_3}}]]
2023-06-04 02:03:07.449145: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_3' with dtype float and shape [?,1,50301]
	 [[{{node input_3}}]]
2023-06-04 02:03:07.454585: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,1,50301]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: model_chi_square/assets


INFO:tensorflow:Assets written to: model_chi_square/assets


In [6]:
x = np.reshape(np.load("anova.npy"), (N << 1, 1, 50301))
cat, dog = x[:N], x[N:]

x_train = np.concatenate([cat[:N_TRAIN], dog[:N_TRAIN]])
x_val = np.concatenate([cat[N_TRAIN:N_TRAIN+N_VAL], dog[N_TRAIN:N_TRAIN+N_VAL]])
x_test = np.concatenate([cat[-N_TEST:], dog[-N_TEST:]])

model = Sequential([
    InputLayer(input_shape=(1, 50301)),
    # Conv1D(1024, 5, padding='same', activation='relu'),
    # Conv1D(512, 11, padding='same', activation='relu'),
    # Conv1D(256, 17, padding='same', activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.optimizers.Adam(0.001),
              loss=tf.keras.losses.binary_crossentropy,
              metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

model.summary()

model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=100, batch_size=500,
          callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)])

print(model.evaluate(x_test, y_test, batch_size=500, return_dict=True))

model.save("model_anova")

del x, cat, dog, x_train, x_val, x_test, model

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 1, 1)              50302     
                                                                 
Total params: 50,302
Trainable params: 50,302
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
40/40 [==============================] - 2s 48ms/step - loss: 0.7011 - binary_accuracy: 0.6091 - precision_3: 0.6076 - recall_3: 0.6163 - val_loss: 0.5832 - val_binary_accuracy: 0.7020 - val_precision_3: 0.7919 - val_recall_3: 0.5480
Epoch 2/100
40/40 [==============================] - 0s 12ms/step - loss: 0.5502 - binary_accuracy: 0.7286 - precision_3: 0.7244 - recall_3: 0.7381 - val_loss: 0.5366 - val_binary_accuracy: 0.7370 - val_precision_3: 0.7384 - val_recall_3: 0.7340
Epoch 3/100
40/40 [==============================] - 1s 13ms/step - loss: 0.5204 - bin

2023-06-04 02:03:36.202890: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_4' with dtype float and shape [?,1,50301]
	 [[{{node input_4}}]]
2023-06-04 02:03:36.207951: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_4' with dtype float and shape [?,1,50301]
	 [[{{node input_4}}]]
2023-06-04 02:03:36.213965: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,1,50301]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: model_anova/assets


INFO:tensorflow:Assets written to: model_anova/assets
